# Introduction to MNE-NIRS and MNE-Python 

Useful links:

Documentation:
* MNE-Python: https://mne.tools
* MNE-NIRS: https://mne.tools/mne-nirs
* Nilearn: https://nilearn.github.io

Code:
* MNE-Python: https://github.com/mne-tools/mne-python
* MNE-NIRS: https://github.com/mne-tools/mne-nirs
* Nilearn: https://github.com/nilearn/nilearn

Community:
* User forum: https://mne.discourse.group/
* Chat room: https://discord.com/invite/rKfvxTuATa

Tutorials:
* All fNIRS tutorials: https://mne.tools/mne-nirs/master/auto_examples/index.html

## Outline

This tutorial will cover the following:

* What is MNE and MNE-NIRS?
* Where can I get help using MNE?
* The basics
  * Python basics
  * How to load NIRx data
  * How to view raw data
  * How to manipulate data
    * How to pick channels
    * How to crop data
* Basic signal processing
  * Conversion to haemoglobin
  * Data quality metrics
* Waveform analysis
* Summary statistics

### Webinar 2: Advanced topics
What should we discuss?
* GLM analysis
* Group level waveform analysis
* Group level GLM analysis
* Cortical projection
* ...
* What do you want to see?

## Python basics

### Packages and functions

In [ ]:
# Importing a package
import mne
import mne_nirs

In [ ]:
# Other ways to import a package
import numpy as np

In [ ]:
# Import just a single function from a package
from itertools import compress

In [ ]:
# Running a function
np.zeros(4)

In [ ]:
# Set defaults using a function
mne.viz.set_3d_backend("notebook")

In [ ]:
%matplotlib inline

## How to load NIRx data

In [ ]:
# Import required libraries
import os
import matplotlib.pyplot as plt

In [ ]:
# Download example data and report the path to data
fnirs_data_folder = mne.datasets.fnirs_motor.data_path()
fnirs_data_folder

In [ ]:
# Get path for just the first participant
fname = f'{fnirs_data_folder}/Participant-1'

In [ ]:
# Load the NIRx data
raw_intensity = mne.io.read_raw_nirx(fname, verbose=True, preload=True)

In [ ]:
raw_intensity

## How to view your data

In [ ]:
raw_intensity.plot_sensors();

In [ ]:
events, event_dict = mne.events_from_annotations(raw_intensity, verbose=False)

In [ ]:
event_dict = {'Control': 1, 'Tapping/Left': 4, 'Tapping/Right': 3, 'ExperimentEnds': 2}

In [ ]:
plt.rcParams["figure.figsize"] = (10, 6) # (w, h)
mne.viz.plot_events(events, event_id=event_dict, sfreq=raw_intensity.info['sfreq']);

## Alternative loading of data

In [ ]:
# NIRSport2 can export as SNIRF which can be read using
# Example of what an error looks like

# mne.io.read_raw_snirf("fake+path")

In [ ]:
# Or the best option is to use MNE-BIDS if your data is formatted correctly
from mne_bids import BIDSPath, read_raw_bids
from mne_nirs.datasets import fnirs_motor_group

bids_path = BIDSPath(subject="01", task="tapping",
                     root=fnirs_motor_group.data_path(),
                     datatype="nirs", suffix="nirs", extension=".snirf")

raw_intensity = read_raw_bids(bids_path=bids_path, verbose=False)

In [ ]:
plt.rcParams["figure.figsize"] = (10, 10) # (w, h)
raw_intensity.plot(duration=300, n_channels=len(raw_intensity.ch_names));

## Interactive figures (will not be used in introduction webinar)

In [ ]:
# Demonstrate interactive figures

# %matplotlib qt
# raw_intensity.plot(duration=300, n_channels=len(raw_intensity.ch_names));

In [ ]:
# Revert back for demo
%matplotlib inline

## How to manipulate data

In [ ]:
# Working on a copy
raw = raw_intensity.copy()

In [ ]:
raw

In [ ]:
raw.resample(2)

In [ ]:
raw

In [ ]:
raw_intensity

In [ ]:
raw_intensity.annotations

In [ ]:
raw_intensity.annotations.to_data_frame()

In [ ]:
raw_intensity.annotations.to_data_frame().plot.scatter(x='onset',  y='description')

### Picking channels

In [ ]:
raw.copy().pick(picks=[0, 1, 2, 3])

In [ ]:
plt.rcParams["figure.figsize"] = (6, 6) # (w, h)
raw.copy().pick(picks=[0, 1, 2, 3]).plot_sensors();

In [ ]:
plt.rcParams["figure.figsize"] = (16, 6) # (w, h)
raw.copy().pick(picks=[0, 1, 2, 3]).plot(duration=300);

In [ ]:
raw.copy().pick(picks=[10]).plot(duration=300);

In [ ]:
raw.copy().pick(picks=range(13, 19)).plot(duration=300);

### Cropping data

In [ ]:
raw.copy().crop(tmin=200, tmax=800).plot(duration=30000);

In [ ]:
raw.copy().pick(range(4)).crop(tmin=200, tmax=800).plot(duration=30000);

## Signal Processing

In [ ]:
raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)

In [ ]:
raw_od

In [ ]:
raw_od.copy().pick(range(6)).ch_names

In [ ]:
# Import required functions
from mne.preprocessing.nirs import beer_lambert_law, optical_density

In [ ]:
raw_haemo = beer_lambert_law(raw_od)

In [ ]:
raw_haemo

In [ ]:
raw_haemo.copy().pick(range(6)).ch_names

In [ ]:
# We can now pick based on type
raw_haemo.copy().pick("hbo").ch_names

In [ ]:
plt.rcParams["figure.figsize"] = (16, 10) # (w, h)
raw_haemo.plot(duration=300, n_channels=len(raw_haemo.ch_names), clipping=None);

In [ ]:
plt.rcParams["figure.figsize"] = (16, 8) # (w, h)
raw_haemo.copy().pick("hbo").plot(duration=300, n_channels=len(raw_haemo.ch_names), clipping=None);

In [ ]:
# Changing the scale
raw_haemo.copy().pick("hbo").plot(duration=300, n_channels=len(raw_haemo.ch_names), clipping=None, scalings=dict(hbo=5e-5));

## Data Quality

In [ ]:
sci = mne.preprocessing.nirs.scalp_coupling_index(raw_od)
sci

In [ ]:
fig, ax = plt.subplots()
ax.hist(sci)
ax.set(xlabel='Scalp Coupling Index', ylabel='Count', xlim=[0, 1])

In [ ]:
# Manual setting of bad channels
raw_od.info['bads'] = ['S1_D9 760', 'S1_D9 850']

In [ ]:
raw_od

In [ ]:
raw_od.copy().pick(range(20)).plot(duration=300, n_channels=len(raw_od.ch_names), clipping=None);

In [ ]:
plt.rcParams["figure.figsize"] = (6, 6) # (w, h)
raw_od.plot_sensors();

In [ ]:
# Run sci just on first 30 seconds of data
sci = mne.preprocessing.nirs.scalp_coupling_index(raw_od.copy().crop(tmax=30))

In [ ]:
from mne_nirs.preprocessing import peak_power, scalp_coupling_index_windowed
from mne_nirs.visualisation import plot_timechannel_quality_metric

_, scores, times = scalp_coupling_index_windowed(raw_od, time_window=60)
plot_timechannel_quality_metric(raw_od, scores, times, threshold=0.7,
                                title="Scalp Coupling Index "
                                      "Quality Evaluation");

## Waveform analysis

### Preprocessing

In [ ]:
raw_intensity = read_raw_bids(bids_path=bids_path, verbose=False)

raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)
raw_haemo = mne.preprocessing.nirs.beer_lambert_law(raw_od)

In [ ]:
raw_haemo = mne_nirs.channels.get_long_channels(raw_haemo)

In [ ]:
raw_haemo.plot_psd(average=True);

### Filtering

In [ ]:
plt.rcParams["figure.figsize"] = (16, 14) # (w, h)
raw_haemo.plot_psd(average=False);

In [ ]:
raw_haemo = raw_haemo.filter(0.05, 0.7, h_trans_bandwidth=0.2, l_trans_bandwidth=0.02)

In [ ]:
raw_haemo.plot_psd(average=True);

### Epoching

In [ ]:
events, event_id = mne.events_from_annotations(raw_haemo)

In [ ]:
plt.rcParams["figure.figsize"] = (16, 6) # (w, h)

fig = mne.viz.plot_events(events, event_id=event_id, sfreq=raw_haemo.info['sfreq'])
fig.subplots_adjust(right=0.7)  # make room for the legend

In [ ]:
reject_criteria = dict(hbo=80e-6)
tmin, tmax = -5, 15

epochs = mne.Epochs(raw_haemo, events, event_id=event_id,
                    tmin=tmin, tmax=tmax, preload=True,
                    reject=reject_criteria, reject_by_annotation=True)

In [ ]:
plt.rcParams["figure.figsize"] = (16, 8) # (w, h)
epochs.plot_drop_log();

In [ ]:
epochs

In [ ]:
epochs['Tapping']

In [ ]:
epochs['Tapping/Right']

In [ ]:
epochs['Tapping/Right'].average()

In [ ]:
epochs['Tapping/Right'].average().plot();

In [ ]:
evoked = epochs['Tapping/Right'].average()

### Plotting

In [ ]:
evoked.plot();

In [ ]:
evoked.pick(picks='hbo').plot_joint(times=[-2, 0, 3, 6, 9, 12], topomap_args=dict(extrapolate='local'));

In [ ]:
evoked_dict = {'Tapping/HbO': epochs['Tapping'].average(picks='hbo').rename_channels(lambda x: x[:-4])}

mne.viz.plot_compare_evokeds(evoked_dict, combine="mean")

In [ ]:
evoked_dict

In [ ]:
evoked_dict = {'Tapping/HbO': epochs['Tapping'].average(picks='hbo'),
               'Tapping/HbR': epochs['Tapping'].average(picks='hbr'),
               'Control/HbO': epochs['Control'].average(picks='hbo'),
               'Control/HbR': epochs['Control'].average(picks='hbr')}
evoked_dict

In [ ]:
for condition in evoked_dict:
    evoked_dict[condition].rename_channels(lambda x: x[:-4])

color_dict = dict(HbO='r', HbR='b')
styles_dict = dict(Control=dict(linestyle='dashed'))

mne.viz.plot_compare_evokeds(evoked_dict, combine="mean", colors=color_dict, styles=styles_dict)

In [ ]:
plt.rcParams["figure.figsize"] = (10, 8) # (w, h)
epochs['Tapping'].copy().pick("hbo").plot_image(combine='mean', vmin=-30, vmax=30,
                                                ts_args=dict(ylim=dict(hbo=[-15, 15],
                                                                       hbr=[-15, 15])))

# Statistical Summary

In [ ]:
epochs["Tapping"]

In [ ]:
import pandas as pd

results = pd.DataFrame()

for epoch in epochs["Tapping"].copy().pick("hbo").crop(tmin=4, tmax=6):

    results = results.append({"Value": epoch.mean() * 1e6, 
                              "Condition": "Tapping"}, 
                             ignore_index=True)

In [ ]:
results.tail()

In [ ]:
for epoch in epochs["Control"].copy().pick("hbo").crop(tmin=4, tmax=6):

    results = results.append({"Value": epoch.mean() * 1e6,
                              "Condition": "Control"},
                             ignore_index=True)

In [ ]:
results["ID"] = "P01"
results.to_csv("WaveformResults.csv")  # Now you can analyse this in your favorite stats program

In [ ]:
results

# Statistical comparison of conditions

In [ ]:
import dabest
analysis_of_long_df = dabest.load(results, idx=("Control", "Tapping"), x="Condition", y="Value")

In [ ]:
analysis_of_long_df

In [ ]:
analysis_of_long_df.mean_diff.plot(swarm_label="Oxyhaemoglobin (µM)\nEpoched value [4-6 seconds]");

In [ ]:
analysis_of_long_df.mean_diff

In [ ]:
analysis_of_long_df.mean_diff.statistical_tests

## Traditional Stats

In [ ]:
from scipy import stats

In [ ]:
results_tapping = results.query('Condition == "Tapping"').Value
results_control = results.query('Condition == "Control"').Value

In [ ]:
stats.ttest_1samp(results_tapping, 0.0)

In [ ]:
stats.ttest_1samp(results_control, 0.0)